In [23]:
import pandas as pd
import json
pd.options.display.float_format = '{:,}'.format

In [34]:
with open("data/response.json", 'rt', encoding='utf-8') as f:
  data = json.load(f)
  
product_info_data = pd.json_normalize(data, record_path=['Grid_20171128000000000572_1','row'])
product_info_data = product_info_data.drop(['M_DISTCTNS_ITM', 'PRDLST_CL', 'MTC_NM', 'MAIN_SPCIES_NM', 'EFFECT', 'PURCHASE_MTH', 'COOK_MTH', 'TRT_MTH', 'URL', 'IMG_URL', 'REGIST_DE'], axis=1)
product_info_data['M_DISTCTNS'] = product_info_data['M_DISTCTNS'].apply(lambda x: x.replace("월", ""))
product_info_data['PRDCTN__ERA'] = product_info_data["PRDCTN__ERA"].apply(lambda x: x.replace("월", ""))

In [6]:
display(product_info_data)

,ROW_NUM,IDNTFC_NO,PRDLST_NM,M_DISTCTNS,PRDCTN__ERA
0,1,227366,산마늘,4,3~5
1,2,227365,울외,6,3~7
2,3,227364,밤콩,7,7
3,4,227363,노각,4,4~7
4,5,227362,삼채,6,6
...,...,...,...,...,...
177,178,219814,당근,12,10~12
178,179,219813,브로콜리,12,9~12
179,180,219812,콜라비,12,11~12
180,181,219811,콜리플라워,12,9~12


In [35]:
product_code_data = pd.read_csv('data/productname.csv')
display(product_code_data)

,PRODUCT_CODE,PRODUCT_NAME
0,111,쌀
1,112,찹쌀
2,141,콩
3,142,팥
4,143,녹두
...,...,...
80,426,건포도
81,427,건블루베리
82,428,망고
83,430,아보카도


In [36]:
product_info = pd.merge(left=product_code_data, right=product_info_data,how="left", left_on='PRODUCT_NAME', right_on='PRDLST_NM')
product_info = product_info.drop(['IDNTFC_NO', 'PRDLST_NM', 'ROW_NUM'], axis=1)
display(product_info)

,PRODUCT_CODE,PRODUCT_NAME,M_DISTCTNS,PRDCTN__ERA
0,111,쌀,10,9~10
1,112,찹쌀,9,9~10
2,141,콩,NaN,NaN
3,142,팥,9,9~10
4,143,녹두,10,9~10
...,...,...,...,...
80,426,건포도,NaN,NaN
81,427,건블루베리,NaN,NaN
82,428,망고,NaN,NaN
83,430,아보카도,NaN,NaN


In [8]:
monthly_20 = pd.read_csv("data/monthly20.01-12.csv", encoding="utf-8")
monthly_21 = pd.read_csv("data/monthly21.01-05.csv", encoding="utf-8")
monthly_22 = pd.read_csv("data/monthly2022.01-05.csv", encoding="utf-8")

In [12]:
monthly_20 = monthly_20[monthly_20['BASE_MM'] == 3]
monthly_21 = monthly_21[monthly_21['BASE_MM'] == 3]
monthly_22 = monthly_22[monthly_22['BASE_MM'] == 3]

In [15]:
concated_monthly = pd.concat([monthly_20, monthly_21, monthly_22], ignore_index=True)

In [40]:
avg_base_cost = concated_monthly.groupby('AGFD_PDLT_NM')['PDTN_QY'].agg(['mean']).astype(int)

In [32]:
display(avg_base_cost)

,mean
AGFD_PDLT_NM,
가지,699244
감,166024
감귤,525670
감자,1021321
갓,4259
...,...
풋옥수수,34360
피망,4326
호박,795661


In [37]:
product_info = pd.merge(left=product_info, right=avg_base_cost,how="left", left_on='PRODUCT_NAME', right_on='AGFD_PDLT_NM')


In [41]:
display(product_info)

,PRODUCT_CODE,PRODUCT_NAME,M_DISTCTNS,PRDCTN__ERA,mean
0,111,쌀,10,9~10,NaN
1,112,찹쌀,9,9~10,NaN
2,141,콩,NaN,NaN,NaN
3,142,팥,9,9~10,NaN
4,143,녹두,10,9~10,NaN
...,...,...,...,...,...
80,426,건포도,NaN,NaN,NaN
81,427,건블루베리,NaN,NaN,NaN
82,428,망고,NaN,NaN,NaN
83,430,아보카도,NaN,NaN,NaN


In [42]:
non_distinct_product = product_info[product_info.notna().all(axis=1)]
distinct_product = product_info[product_info.isna().any(axis=1)]

In [43]:
display(non_distinct_product)

,PRODUCT_CODE,PRODUCT_NAME,M_DISTCTNS,PRDCTN__ERA,mean
6,151,고구마,9,9~10,"1,397,962.0"
7,152,감자,7,6~10,"1,021,321.0"
10,162,보리,6,6,"8,616.0"
13,114,기장,9,8~10,"7,539.0"
14,211,배추,11,11~12,"755,930.0"
15,212,양배추,5,5~7,"4,862,197.0"
17,280,브로콜리,12,9~12,"71,967.0"
18,213,시금치,2,11~2,"490,876.0"
19,214,상추,5,5~8,"218,760.0"
21,216,갓,10,9~12,"4,259.0"


In [45]:
display(distinct_product)

,PRODUCT_CODE,PRODUCT_NAME,M_DISTCTNS,PRDCTN__ERA,mean
0,111,쌀,10,9~10,NaN
1,112,찹쌀,9,9~10,NaN
2,141,콩,NaN,NaN,NaN
3,142,팥,9,9~10,NaN
4,143,녹두,10,9~10,NaN
5,144,메밀,10,10~11,NaN
8,161,귀리,9,9~10,NaN
9,113,혼합곡,NaN,NaN,NaN
11,163,수수,10,9~10,NaN
12,164,율무,9,9~10,NaN
